In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from pandasql import sqldf

In [ ]:
file_path = 'freMTPL2freq.csv'

try:
    df = pd.read_csv(file_path)
    print("Dataset berhasil dimuat!")
    print(f"Jumlah baris: {df.shape[0]}, Jumlah kolom: {df.shape[1]}")
except FileNotFoundError:
    print(f"Error: File tidak ditemukan.")
    df = None 

In [ ]:
print("\n5 Baris Pertama Data")
print(df.head())

print("\nInformasi Umum Data")
df.info()

In [ ]:
print("\nStatistik Deskriptif Data")
print(df.describe(include='all'))

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['ClaimNb'], bins=range(int(df['ClaimNb'].max()) + 2), kde=False) 
plt.title('Distribusi Jumlah Klaim (ClaimNb)')
plt.xlabel('Jumlah Klaim per Polis')
plt.ylabel('Frekuensi')
plt.xticks(range(int(df['ClaimNb'].max()) + 1)) 
plt.grid(axis='y', alpha=0.75)
plt.show()

print(f"\nValue Counts untuk ClaimNb:\n{df['ClaimNb'].value_counts().sort_index()}")
print(f"\nPersentase polis tanpa klaim: {df[df['ClaimNb'] == 0].shape[0] / df.shape[0] * 100:.2f}%")

In [ ]:
numeric_features = ['VehAge', 'DrivAge', 'BonusMalus', 'Density', 'Exposure']

plt.figure(figsize=(18, 12)) 
for i, col in enumerate(numeric_features):
    plt.subplot(2, 3, i + 1) 
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribusi {col}')
    plt.xlabel(col)
    plt.ylabel('Frekuensi')
    plt.grid(axis='y', alpha=0.75)
plt.tight_layout()
plt.show()

In [ ]:
categorical_features = ['Area', 'VehBrand', 'VehGas', 'Region']

plt.figure(figsize=(18, 12))
for i, col in enumerate(categorical_features):
    plt.subplot(2, 2, i + 1)
    sns.countplot(y=df[col], order=df[col].value_counts().index, palette='viridis')
    plt.title(f'Distribusi {col}')
    plt.xlabel('Frekuensi')
    plt.ylabel(col)
    plt.grid(axis='x', alpha=0.75)
plt.tight_layout()
plt.show()

In [ ]:
if (df['Exposure'] == 0).any():
    print("Warning: Ada nilai Exposure 0 yang dapat menyebabkan error pembagian.")
    df['Exposure'] = df['Exposure'].replace(0, 1e-6)

df['ClaimRate'] = df['ClaimNb'] / df['Exposure']
print("\nDeskripsi ClaimRate")
print(df['ClaimRate'].describe())

plt.figure(figsize=(10, 6))
sns.histplot(df['ClaimRate'], bins=50, kde=True)
plt.title('Distribusi Claim Rate')
plt.xlabel('Claim Rate (Klaim per Tahun Eksposur)')
plt.ylabel('Frekuensi')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.75)
plt.show()

print(f"\nValue Counts untuk ClaimRate > 0:\n{df[df['ClaimRate'] > 0]['ClaimRate'].value_counts().sort_index().head(10)}")

In [ ]:
bins_drivage = [18, 25, 35, 45, 55, 65, 100] 
labels_drivage = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']
df['DrivAgeGroup'] = pd.cut(df['DrivAge'], bins=bins_drivage, labels=labels_drivage, right=False)
print("\nDrivAgeGroup Value Counts")
print(df['DrivAgeGroup'].value_counts().sort_index())


bins_vehage = [0, 5, 10, 15, 20, df['VehAge'].max() + 1]
labels_vehage = ['0-4', '5-9', '10-14', '15-19', '20+']
df['VehAgeGroup'] = pd.cut(df['VehAge'], bins=bins_vehage, labels=labels_vehage, right=False)
print("\nVehAgeGroup Value Counts")
print(df['VehAgeGroup'].value_counts().sort_index())

bins_bonusmalus = [50, 51, 60, 100, 150, df['BonusMalus'].max() + 1]
labels_bonusmalus = ['Bonus (50)', 'Bonus (51-59)', 'Bonus (60-99)', 'Malus (100-149)', 'Malus (150+)']
df['BonusMalusGroup'] = pd.cut(df['BonusMalus'], bins=bins_bonusmalus, labels=labels_bonusmalus, right=False)
print("\nBonusMalusGroup Value Counts")
print(df['BonusMalusGroup'].value_counts().sort_index())

In [ ]:
categorical_features_for_claim_rate = ['Area', 'VehBrand', 'VehGas', 'Region', 'DrivAgeGroup', 'VehAgeGroup', 'BonusMalusGroup']

plt.figure(figsize=(18, 20)) 

for i, col in enumerate(categorical_features_for_claim_rate):
    plt.subplot(4, 2, i + 1) 
    claim_rate_by_category = df.groupby(col)['ClaimRate'].mean().sort_values(ascending=False)
    sns.barplot(x=claim_rate_by_category.values, y=claim_rate_by_category.index, palette='viridis')
    plt.title(f'Rata-rata Claim Rate Berdasarkan {col}')
    plt.xlabel('Rata-rata Claim Rate')
    plt.ylabel(col)
    plt.grid(axis='x', alpha=0.75)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='VehPower', y='ClaimRate', data=df)
plt.title('Distribusi Claim Rate Berdasarkan VehPower')
plt.xlabel('Kekuatan Kendaraan (VehPower)')
plt.ylabel('Claim Rate')
plt.grid(axis='y', alpha=0.75)
plt.show()

# jika hanya rata-rata:
# plt.figure(figsize=(10, 6))
# claim_rate_by_power = df.groupby('VehPower')['ClaimRate'].mean().sort_index()
# sns.barplot(x=claim_rate_by_power.index, y=claim_rate_by_power.values, palette='viridis')
# plt.title('Rata-rata Claim Rate Berdasarkan VehPower')
# plt.xlabel('Kekuatan Kendaraan (VehPower)')
# plt.ylabel('Rata-rata Claim Rate')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

In [ ]:
region_risk = df.groupby('Region')['ClaimRate'].mean().sort_values()
top_5_risky_regions = region_risk.tail(5).index
bottom_5_risky_regions = region_risk.head(5).index
selected_regions = top_5_risky_regions.union(bottom_5_risky_regions)

df_filtered_regions = df[df['Region'].isin(selected_regions)]

pivot_age_region = pd.pivot_table(
    df_filtered_regions,
    values='ClaimRate',
    index='DrivAgeGroup',
    columns='Region',
    aggfunc='mean'
)

plt.figure(figsize=(14, 8))
sns.heatmap(
    pivot_age_region,
    annot=True,          
    fmt=".3f",          
    cmap='coolwarm',     
    linewidths=.5
)
plt.title('Heatmap Rata-rata Claim Rate: Interaksi Usia Pengemudi dan Region', fontsize=16)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Grup Usia Pengemudi', fontsize=12)
plt.show()

In [ ]:
top_5_brands = df['VehBrand'].value_counts().nlargest(5).index
df_filtered_brands = df[df['VehBrand'].isin(top_5_brands)]

plt.figure(figsize=(14, 8))
sns.barplot(
    data=df_filtered_brands,
    x='VehAgeGroup',
    y='ClaimRate',
    hue='VehBrand', 
    palette='viridis'
)
plt.title('Rata-rata Claim Rate: Interaksi Usia dan Merek Kendaraan', fontsize=16)
plt.xlabel('Grup Usia Kendaraan', fontsize=12)
plt.ylabel('Rata-rata Claim Rate', fontsize=12)
plt.legend(title='Merek Kendaraan')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
df['DensityGroup'] = pd.qcut(
    df['Density'],
    q=4,
    labels=['Rural', 'Suburban', 'Urban', 'Very Urban']
)

density_claim_rate = df.groupby('DensityGroup')['ClaimRate'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(
    x=density_claim_rate.index,
    y=density_claim_rate.values,
    palette='magma'
)
plt.title('Rata-rata Claim Rate Berdasarkan Kepadatan Area', fontsize=16)
plt.xlabel('Grup Kepadatan Penduduk', fontsize=12)
plt.ylabel('Rata-rata Claim Rate', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
insurance_claims_table = df.copy()
pysqldf = lambda q: sqldf(q, globals())

query1 = """
SELECT
    DrivAgeGroup,
    COUNT(IDpol) AS total_policies,
    AVG(ClaimRate) AS average_claim_rate
FROM
    insurance_claims_table
GROUP BY
    DrivAgeGroup
ORDER BY
    DrivAgeGroup;
"""
print("Rata-rata Claim Rate per Grup Usia Pengemudi")
result1 = pysqldf(query1)
print(result1)




In [ ]:
query2 = """
SELECT
    Region,
    AVG(ClaimRate) AS average_claim_rate
FROM
    insurance_claims_table
GROUP BY
    Region
ORDER BY
    average_claim_rate DESC
LIMIT 5;
"""
print("\nTop 5 Region Paling Berisiko")
result2 = pysqldf(query2)
print(result2)



In [ ]:
query3 = """
SELECT
    DensityGroup,
    AVG(ClaimRate) AS average_claim_rate
FROM
    insurance_claims_table
WHERE
    DensityGroup IS NOT NULL
GROUP BY
    DensityGroup
ORDER BY
    average_claim_rate DESC;
"""
print("\nRata-rata Claim Rate per Kategori Kepadatan")
result3 = pysqldf(query3)
print(result3)



In [ ]:
query4 = """
SELECT
    Region,
    AVG(ClaimRate) AS avg_claim_rate_young_drivers
FROM
    insurance_claims_table
WHERE
    DrivAgeGroup = '18-24' AND
    Region IN ('R21', 'R93', 'R72', 'R82', 'R52') -- Ganti dengan top 5 region dari hasil Anda
GROUP BY
    Region
ORDER BY
    avg_claim_rate_young_drivers DESC;
"""
print("\nAnalisis Interaksi Risiko Tinggi ---")
result4 = pysqldf(query4)
print(result4)

In [ ]:
columns_for_dashboard = [
    'ClaimNb', 'Exposure', 'ClaimRate', 'Area', 'VehPower', 'VehAge', 'DrivAge',
    'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region', 'DrivAgeGroup',
    'VehAgeGroup', 'BonusMalusGroup', 'DensityGroup'
]

df_for_dashboard = df[columns_for_dashboard]
output_path = 'insurance_claim_data_for_dashboard.csv'
df_for_dashboard.to_csv(output_path, index=False)

print(f"File: '{output_path}'")